In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

# Define paths to your training, testing, and validation data
train_data_dir = r'D:\Thesis\train'
test_data_dir = r'D:\Thesis\test'
valid_data_dir = r'D:\Thesis\validate'

# Data preprocessing with augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)

# Set batch size and image dimensions
batch_size = 32
img_height, img_width = 224, 224

# Generate augmented data for training and validation sets
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

valid_generator = valid_datagen.flow_from_directory(
    valid_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

# Load the pre-trained VGG16 model without the top classification layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Add custom classification layers with Dropout for regularization
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)  # Adding dropout for regularization
predictions = Dense(len(train_generator.class_indices), activation='softmax')(x)

# Compile the model
model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=0.01), loss='categorical_crossentropy', metrics=['accuracy'])


# Implement learning rate reduction and early stopping
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model with callbacks
epochs = 50
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=valid_generator,
    validation_steps=valid_generator.samples // batch_size,
    callbacks=[reduce_lr, early_stop]
)

# Evaluate the model on the test set
evaluation = model.evaluate(test_generator)
print(f"Test Accuracy: {evaluation[1] * 100:.2f}%")


Found 1576 images belonging to 333 classes.
Found 364 images belonging to 333 classes.
Found 380 images belonging to 333 classes.
Epoch 1/50
49/49 [==============================] - 14s 273ms/step - loss: 5.3270 - accuracy: 0.0045 - val_loss: 4.9501 - val_accuracy: 0.0256 - lr: 0.0100
Epoch 2/50
49/49 [==============================] - 13s 269ms/step - loss: 4.8084 - accuracy: 0.0207 - val_loss: 4.6710 - val_accuracy: 0.0256 - lr: 0.0100
Epoch 3/50
49/49 [==============================] - 13s 269ms/step - loss: 4.6672 - accuracy: 0.0207 - val_loss: 4.5194 - val_accuracy: 0.0597 - lr: 0.0100
Epoch 4/50
49/49 [==============================] - 13s 269ms/step - loss: 4.5051 - accuracy: 0.0402 - val_loss: 4.4107 - val_accuracy: 0.0852 - lr: 0.0100
Epoch 5/50
49/49 [==============================] - 13s 273ms/step - loss: 4.4228 - accuracy: 0.0356 - val_loss: 4.2706 - val_accuracy: 0.0994 - lr: 0.0100
Epoch 6/50
49/49 [==============================] - 13s 269ms/step - loss: 4.3024 - accura

KeyboardInterrupt: 